# 基于SSD实现目标检测

## SSD简介

目标检测的算法按照原理主要分为two-stage和one-stage两大类，其中：

**two-stage**：以R-CNN系列为代表，这类方法通常包括两个部分，第一部分先使用selective search、卷积神经网络等筛选出一些proposal boxes，然后第二部分再对这些proposal boxes进行分类和回归。这就相当于进行了两次分类和回归，因此检测的准确率较高，但是可想而知检测的速度也就比较慢了。

**one-stage**：以YOLO为代表，这类方法的主要思路就是在图片的不同位置进行密集采样，然后使用CNN网络提取特征并直接进行分类和回归，整个过程只要一步就可完成。这种方法的优势是检测速度快，但是检测的准确率却不是很高。

SSD目标检测算法出现在Faster-RCNN和YOLOV1算法之后，融合了二者的优点的one-stage算法。速度比Faster-RCNN更快，比YOLOV1算法准确率更高。

## 算法设计理念

SSD和YOLO一样都是采用一个CNN网络来进行检测，但是却采用了多尺度的特征图，下面将SSD核心设计理念总结为以下三点：

### 采用多尺度特征图用于检测

所谓多尺度采用大小不同的特征图，CNN网络一般前面的特征图比较大，后面会逐渐采用stride=2的卷积或者pool来降低特征图大小，一个比较大的特征图和一个比较小的特征图，它们都用来做检测。这样做的好处是比较大的特征图来用来检测相对较小的目标，而小的特征图负责检测大目标，如图1所示，8x8的特征图可以划分更多的单元，但是其每个单元的先验框尺度比较小。

![图1](./images/feature_map.png)
<center><i>图1</i></center>

### 采用卷积进行检测

与Yolo最后采用全连接层不同，SSD直接采用卷积对不同的特征图来进行提取检测结果。对于形状为$m \times n \times p$的特征图，只需要采用$3 \times 3 \times p$这样比较小的卷积核得到检测值。

### 设置先验框

在Yolo中，每个单元预测多个边界框，但是其都是相对这个单元本身（正方块），但是真实目标的形状是多变的，Yolo需要在训练过程中自适应目标的形状。而SSD借鉴了Faster R-CNN中anchor的理念，每个单元设置尺度或者长宽比不同的先验框，预测的边界框（bounding boxes）是以这些先验框为基准的，在一定程度上减少训练难度。一般情况下，每个单元会设置多个先验框，其尺度和长宽比存在差异，如图2所示，可以看到每个单元使用了4个不同的先验框，图片中猫和狗分别采用最适合它们形状的先验框来进行训练，后面会详细讲解训练过程中的先验框匹配原则。

![图2](./images/cat_dog.jpg)
<center><i>图2</i></center>

SSD的检测值也与Yolo不太一样。对于每个单元的每个先验框，其都输出一套独立的检测值，对应一个边界框，主要分为两个部分。第一部分是各个类别的置信度或者评分，值得注意的是SSD将背景也当做了一个特殊的类别，如果检测目标共有 [公式] 个类别，SSD其实需要预测 [公式] 个置信度值，其中第一个置信度指的是不含目标或者属于背景的评分。在预测过程中，置信度最高的那个类别就是边界框所属的类别，特别地，当第一个置信度值最高时，表示边界框中并不包含目标。第二部分就是边界框的location，包含4个值$(cx, cy, w, h)$，分别表示边界框的中心坐标以及宽高。但是真实预测值其实只是边界框相对于先验框的转换值(paper里面说是offset)。先验框位置用$d = (d^{cx}, d^{cy}, d^w, d^h)$表示，其对应边界框用$b = (b^{cx}, b^{cy}, b^w, b^h)$表示，那么边界框的预测值$l$其实是$b$相对于$d$的转换值：

$$l^{cx} = (b^{cx} - d^{cx} / d^w)$$

$$l^{cy} = (b^{cy} - d^{cy} / d^h)$$

$$l^w = \log(b^w / d^w)$$

$$l^h = \log(b^h / d^h)$$

习惯上，我们称上面这个过程为边界框的编码（encode），预测时，你需要反向这个过程，即进行解码（decode），从预测值$l$中得到边界框的真实位置$b$：

$$b^{cx} = d^w \times l^{cx} + d^{cx}$$

$$b^{cy} = d^h \times l^{cy} + d^{cy}$$

$$b^w = d^w \times e^{lw}$$

$$b^h = d^h \times e^{lh}$$

综上所述，对于一个大小$m \times n$的特征图，共有$mn$个单元，每个单元设置的先验框数目记为$k$，那么每个单元共需要$(c + 4)k$个预测值，所有的单元共需要$(c + 4)kmn$个预测值，由于SSD采用卷积做检测，所以就需要$(c + 4)k$个卷积核完成这个特征图的检测过程。

## 模型解析

### 模型架构

SSD论文采用VGG16作为基础模型，然后在VGG16的基础上新增了卷积层来获得更多的特征图以用于检测。可以明显看到SSD利用了多尺度的特征图做检测。模型的输入图片大小是300x300。

![图3](./images/ssd_vgg.png)
<center><i>图3</i></center>

但是本案例使用MobileNetV2作为基础模型用于目标检测，总共提取了6个特征图用于目标检测，其中从MobileNetV2网络中提取了两个特征图，特征大小分别为$19 \times 19$，$10 \times 10$，代码如下所示：

In [ ]:
from mindspore import nn

from mindvision.classification.models.blocks import ConvNormActivation
from mindvision.classification.models.utils import make_divisible
from mindvision.classification.models.backbones import MobileNetV2


class SSDMobileNetV2(nn.Cell):
    """
    MobileNet V2 backbone.
    """

    def __init__(self):
        super(SSDMobileNetV2, self).__init__()
        net = MobileNetV2()
        features = net.features
        self.features_1 = features[:14]
        self.features_2 = features[14:]

        input_channel = make_divisible(96 * 1.0, 8)
        hidden_channel = int(round(input_channel * 6))
        self.expand_layer = ConvNormActivation(input_channel, hidden_channel, kernel_size=1, activation=nn.ReLU6)

    def construct(self, x):
        """Forward pass"""
        out = self.features_1(x)
        expand_layer = self.expand_layer(out)
        out = self.features_2(out)

        return expand_layer, out

因为一共提取了6个特征图，但是MobileNetv2只提取了两个，所以扩展了4个特征图，特征图大小分别为$5 \times 5$，$3 \times 3$， $2 \times 2$，$1 \times 1$，代码如下所示：

In [ ]:
from typing import List

import mindspore.nn as nn

from mindvision.classification.models.backbones.mobilenet_v2 import InvertedResidual


class SSDMobileNetV2Neck(nn.Cell):
    """
    MobileNetV2 as SSD backbone.
    """
    def __init__(self,
                 extras_in_channels: List[int],
                 extras_out_channels: List[int],
                 extras_strides: List[int],
                 extras_ratios: List[float]):
        super(SSDMobileNetV2Neck, self).__init__()
        residual_list = []
        for i in range(2, len(extras_in_channels)):
            residual = InvertedResidual(extras_in_channels[i],
                                        extras_out_channels[i],
                                        stride=extras_strides[i],
                                        expand_ratio=extras_ratios[i],
                                        last_relu=True
                                        )
            residual_list.append(residual)

        self.multi_residual = nn.CellList(residual_list)

    def construct(self, x):
        expand_layer, output = x
        multi_feature = (expand_layer, output)
        feature = output

        for residual in self.multi_residual:
            feature = residual(feature)
            multi_feature += (feature,)

        return multi_feature

## Default box

SSD中的Defalut box和Faster-rcnn中的anchor机制很相似。就是预设一些目标预选框，后续通过softmax分类 + bounding box regression获得真实目标的位置。对于不同尺度的feature map 上使用不同的Default boxes。如上所示，我们选取的feature map包括$19 \times 19$，$10 \times 10$，$5 \times 5$，$3 \times 3$， $2 \times 2$，$1 \times 1$，每个feature map上的每一个点分别生成$3$，$6$，$6$，$6$，$6$，$6$个Defalut box，即我们总共可以获得1917个box，然后我们将这些box送入NMS模块中，获得最终的检测结果。

以feature map上每个点的中点为中心（offset=0.5），生成一系列同心的Defalut box，使用m(SSD300中m=6)个不同大小的feature map 来做预测，最底层的 feature map的scale值为$min\_scale=0.2$，最高层的为$max\_scale=0.95$，使用不同的ratio值，计算Default box的宽度w和高度h，代码如下所示：

In [ ]:
from typing import List
import math
import itertools
import numpy as np


class GenerateDefaultBoxes:
    """
    Generate Default boxes for SSD.
    """

    def __init__(self,
                 img_shape: int,
                 steps: List[int],
                 max_scale: float,
                 min_scale: float,
                 num_default: List[int],
                 feature_size: List[int],
                 aspect_ratios: List[List[float]]
                 ):
        fk = img_shape[0] / np.array(steps)
        scale_rate = (max_scale - min_scale) / (len(num_default) - 1)
        scales = [min_scale + scale_rate * i for i in range(len(num_default))] + [1.0]
        self.default_boxes = []
        for idx, feature in enumerate(feature_size):
            sk1 = scales[idx]
            sk2 = scales[idx + 1]
            sk3 = math.sqrt(sk1 * sk2)
            if idx == 0 and not aspect_ratios[idx]:
                w, h = sk1 * math.sqrt(2), sk1 / math.sqrt(2)
                all_sizes = [(0.1, 0.1), (w, h), (h, w)]
            else:
                all_sizes = [(sk1, sk1)]
                for aspect_ratio in aspect_ratios[idx]:
                    w, h = sk1 * math.sqrt(aspect_ratio), sk1 / math.sqrt(aspect_ratio)
                    all_sizes.append((w, h))
                    all_sizes.append((h, w))
                all_sizes.append((sk3, sk3))

            for i, j in itertools.product(range(feature), repeat=2):
                for w, h in all_sizes:
                    cx, cy = (j + 0.5) / fk[idx], (i + 0.5) / fk[idx]
                    self.default_boxes.append([cx, cy, w, h])

        def to_tlbr(cx, cy, w, h):
            return cx - w / 2, cy - h / 2, cx + w / 2, cy + h / 2

        # For IoU calculation
        self.default_boxes_tlbr = np.array(tuple(to_tlbr(*i) for i in self.default_boxes), dtype='float32')
        self.default_boxes = np.array(self.default_boxes, dtype='float32')


### Focal Loss

Focal Loss的引入主要是为了解决难易样本数量不均衡的问题。单阶段的目标检测器通常就会产生很多的候选目标，只有极少数是正样本，正负样本数量非常不均衡。我们在计算分类的时候常用的损失为交叉熵，公式如下：

$$
CE=
\begin{cases}
-\log(p),&if\ y = 1 \\
&&(1)\\
-\log(1 - p),&if\ y = 0 \\
\end{cases}
$$

为了解决正负样本不平衡的问题，我们通常会在交叉熵损失函数前面加上一个参数$\alpha$,即：

$$
CE=
\begin{cases}
-\alpha\log(p),&if\ y = 1 \\
&&(2)\\
-(1 - \alpha)\log(1 - p),&if\ y = 0 \\
\end{cases}
$$

尽管$\alpha$平衡了正负样本，但是对难易样本的不平衡没有任何的帮助。而实际上，目标检测中大量的候选目标都是易分样本。这些样本的损失很低，但是由于数量极不平衡，易分样本的数量相对来讲太多，最终主导了总的损失，易分样本（置信度高的样本）对模型提升效果非常小，模型应该主要关注那些难分的样本，这时候Focal Loss就上场了。

一个简单的思想：把高置信度样本的损失再降低一些即可。

$$
FL=
\begin{cases}
-(1 - p)^\gamma\log(p),&if\ y = 1 \\
&&(3)\\
-p^\gamma\log(1 - p),&if\ y = 0 \\
\end{cases}
$$

举个例，$\gamma$取2时，如果$p = 0.968$, $(1 - 0.968)^2\approx0.001$，损失衰减了1000倍。

Focal Loss的最终形式结合了上面的公式(2)。这很好理解，公式(3)解决了难易样本的不平衡，公式(2)解决了正负样本的不平衡，将公式(2)与公式(3)结合使用，同时解决正负难易2个问题。最终的Focal Loss形式如下：

$$
FL=
\begin{cases}
-\alpha(1 - p)^\gamma\log(p),&if\ y = 1 \\
-(1 - \alpha)p^\gamma\log(1 - p),&if\ y = 0 \\
\end{cases}
$$

实验表明$\gamma$取2，$\alpha$取0.25时候效果最佳，代码实现如下：

In [ ]:
import mindspore as ms
import mindspore.ops as ops
import mindspore.nn as nn
from mindspore import Tensor


class SigmoidFocalClassificationLoss(nn.Cell):
    """"
    Sigmoid focal-loss for classification.
    """

    def __init__(self, gamma=2.0, alpha=0.25):
        super(SigmoidFocalClassificationLoss, self).__init__()
        self.sigmoid_cross_entropy = ops.SigmoidCrossEntropyWithLogits()
        self.sigmoid = ops.Sigmoid()
        self.pow = ops.Pow()
        self.onehot = ops.OneHot()
        self.on_value = Tensor(1.0, ms.float32)
        self.off_value = Tensor(0.0, ms.float32)
        self.gamma = gamma
        self.alpha = alpha

    def construct(self, logits, label):
        label = self.onehot(label, ops.shape(logits)[-1], self.on_value, self.off_value)
        sigmoid_cross_entropy = self.sigmoid_cross_entropy(logits, label)
        sigmoid = self.sigmoid(logits)
        label = ops.cast(label, ms.float32)
        p_t = label * sigmoid + (1 - label) * (1 - sigmoid)
        modulating_factor = self.pow(1 - p_t, self.gamma)
        alpha_weight_factor = label * self.alpha + (1 - label) * (1 - self.alpha)
        focal_loss = modulating_factor * alpha_weight_factor * sigmoid_cross_entropy
        return focal_loss


### MultiBox

该层主要包括两部分，一部分是对预测的类别分数进行预测，另一部分是对边界框回归参数进行预测。其中`MultiBox`有两个分支，当为训练模式的时候，分支`construct_train`对预测的值进行损失计算，其中分类使用Focal Loss损失，回归使用SmoothL1Loss损失。当为非训练模式的时候，分支`construct_eval`对预测的类别分数进行Sigmoid处理，对预测的边界框回归参数进行解码处理，得到边界框的真实位置。

In [ ]:
from typing import List, Union
import numpy as np

import mindspore as ms
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor

from mindvision.classification.models.blocks import ConvNormActivation
from mindvision.msdetection.loss.sigmoid_focal_classification_loss import SigmoidFocalClassificationLoss
from mindvision.msdetection.internals.anchor import GenerateDefaultBoxes, GridAnchorGenerator


class FlattenConcat(nn.Cell):
    """
    Concatenate predictions into a single tensor.
    """

    def __init__(self, num_ssd_boxes: int):
        super(FlattenConcat, self).__init__()
        self.num_ssd_boxes = num_ssd_boxes
        self.concat = ops.Concat(axis=1)
        self.transpose = ops.Transpose()

    def construct(self, inputs):
        output = ()
        batch_size = inputs[0].shape[0]
        for x in inputs:
            x = self.transpose(x, (0, 2, 3, 1))
            output += (x.reshape((batch_size, -1)),)
        res = self.concat(output)
        return res.reshape((batch_size, self.num_ssd_boxes, -1))


class MultiBox(nn.Cell):
    """
    Multi-box conv layers. Each multi-box layer contains class conf scores and localization predictions.
    """

    def __init__(self,
                 num_classes: int,
                 extras_out_channels: List[int],
                 num_default: List[int],
                 num_ssd_boxes: int,
                 gamma: float,
                 alpha: float,
                 anchor_generator: Union[GridAnchorGenerator, GenerateDefaultBoxes],
                 prior_scaling: List[float]
                 ):
        super(MultiBox, self).__init__()

        loc_layers = []
        cls_layers = []
        for k, out_channel in enumerate(extras_out_channels):
            loc_layers += [
                nn.SequentialCell([ConvNormActivation(
                    out_channel,
                    out_channel,
                    kernel_size=3,
                    stride=1,
                    groups=out_channel,
                    activation=nn.ReLU6
                ), nn.Conv2d(out_channel, 4 * num_default[k], kernel_size=1, has_bias=True)])
            ]
            cls_layers += [
                nn.SequentialCell([ConvNormActivation(
                    out_channel,
                    out_channel,
                    kernel_size=3,
                    stride=1,
                    groups=out_channel,
                    activation=nn.ReLU6
                ), nn.Conv2d(out_channel, num_classes * num_default[k], kernel_size=1, has_bias=True)])
            ]

        self.multi_loc_layers = nn.CellList(loc_layers)
        self.multi_cls_layers = nn.CellList(cls_layers)
        self.flatten_concat = FlattenConcat(num_ssd_boxes)

        self.less = ops.Less()
        self.tile = ops.Tile()
        self.reduce_sum = ops.ReduceSum()
        self.expand_dims = ops.ExpandDims()
        self.class_loss = SigmoidFocalClassificationLoss(gamma, alpha)
        self.loc_loss = nn.SmoothL1Loss()
        self.activation = ops.Sigmoid()
        self.default_boxes = Tensor(anchor_generator.default_boxes)
        self.prior_scaling_xy = prior_scaling[0]
        self.prior_scaling_wh = prior_scaling[1]
        self.exp = ops.Exp()
        self.concat = ops.Concat(-1)
        self.maximum = ops.Maximum()
        self.minimum = ops.Minimum()

    def construct(self, inputs):
        loc_outputs = ()
        cls_outputs = ()
        for i in range(len(self.multi_loc_layers)):
            loc_outputs += (self.multi_loc_layers[i](inputs[i]),)
            cls_outputs += (self.multi_cls_layers[i](inputs[i]),)
        return self.flatten_concat(loc_outputs), self.flatten_concat(cls_outputs)

    def construct_train(self, features, boxes, labels):
        """Training."""
        pred_loc, pred_label = self.construct(features)
        pred_loc = ops.cast(pred_loc, ms.float32)
        pred_label = ops.cast(pred_label, ms.float32)
        gt_loc = boxes
        gt_label = labels

        num_matched_boxes = []
        for label in labels:
            num_matched_boxes.append(ops.count_nonzero(label))

        mask = ops.cast(self.less(0, gt_label), ms.float32)
        num_matched_boxes = self.reduce_sum(ops.cast(Tensor(num_matched_boxes), ms.float32))
        # Localization Loss
        mask_loc = self.tile(self.expand_dims(mask, -1), (1, 1, 4))
        smooth_l1 = self.loc_loss(pred_loc, gt_loc) * mask_loc
        loss_loc = self.reduce_sum(self.reduce_sum(smooth_l1, -1), -1)

        # Classification Loss
        loss_cls = self.class_loss(pred_label, gt_label)
        loss_cls = self.reduce_sum(loss_cls, (1, 2))
        return self.reduce_sum((loss_cls + loss_loc) / num_matched_boxes)

    def construct_eval(self, features):
        """Eval."""
        pred_loc, pred_label = self.construct(features)
        pred_label = self.activation(pred_label)
        pred_loc = ops.cast(pred_loc, ms.float32)
        pred_label = ops.cast(pred_label, ms.float32)

        default_bbox_xy = self.default_boxes[..., :2]
        default_bbox_wh = self.default_boxes[..., 2:]
        pred_xy = pred_loc[..., :2] * self.prior_scaling_xy * default_bbox_wh + default_bbox_xy
        pred_wh = self.exp(pred_loc[..., 2:] * self.prior_scaling_wh) * default_bbox_wh

        pred_xy_0 = pred_xy - pred_wh / 2.0
        pred_xy_1 = pred_xy + pred_wh / 2.0
        pred_xy = self.concat((pred_xy_0, pred_xy_1))
        pred_xy = self.maximum(pred_xy, 0)
        pred_xy = self.minimum(pred_xy, 1)
        return pred_xy, pred_label

## 模型训练与推理

本案例基于MindSpore-GPU版本，在单GPU卡上完成模型训练和验证。

开始实验之前，请确保本地已经安装了Python环境并安装了[MindSpore Vision套件](https://mindspore.cn/vision/docs/zh-CN/master/index.html)。

案例代码在MindSpore Vision套件中都有API可直接调用，详情可以参考以下链接：https://gitee.com/mindspore/vision 。

### 环境准备与数据读取

可通过<https://cocodataset.org>下载完整的COCO数据集。

这里以下载COCO2017数据集为例，主要下载三个文件：

- 2017 Train images [118K/18GB]：训练过程中使用到的所有图像文件。

- 2017 Val images [5K/1GB]：验证过程中使用到的所有图像文件。

- 2017 Train/Val annotations [241MB]：对应训练集和验证集的标注json文件。

请确保你的数据集路径如以下结构。

```text
.coco2017/
    ├── train2017/
    ├── val2017/
    └── annotations/
```

### 模型训练

MindSpore Vision套件提供了提供了相应的接口，具体代码如下所示。

In [ ]:
import mindspore as ms
import mindspore.nn as nn
from mindspore import context
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

from mindvision.msdetection.dataset import COCODetection
from mindvision.msdetection.models.ssd import ssd_mobilenet_v2
from mindvision.msdetection.models.utils import TrainingWrapper
from mindvision.msdetection.internals.anchor import GenerateDefaultBoxes
from mindvision.msdetection.models.utils import SSDEncoder
from mindvision.engine.callback import LossMonitor
from mindvision.msdetection.dataset.transforms import DetectionDecode, RandomSampleCrop, DetectionResize, \
    DetectionToPercentCoords, AssignGTToDefaultBox, DetectionRandomColorAdjust, DetectionNormalize, DetectionHWC2CHW

data_url = './coco2017/'
ckpt_save_dir = './ssd_ckpt/'
epoch_size = 100


context.set_context(mode=context.GRAPH_MODE,
                    device_target="GPU",
                    enable_graph_kernel=True,
                    graph_kernel_flags="--enable_parallel_fusion --enable_expand_ops=Conv2D")

# Data Pipeline.
anchor_generator = GenerateDefaultBoxes(img_shape=(300, 300),
                                        steps=[16, 32, 64, 100, 150, 300],
                                        max_scale=0.95,
                                        min_scale=0.2,
                                        num_default=[3, 6, 6, 6, 6, 6],
                                        feature_size=[19, 10, 5, 3, 2, 1],
                                        aspect_ratios=[[], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]])
ssd_encoder = SSDEncoder(match_threshold=0.5,
                         prior_scaling=[0.1, 0.2],
                         anchor_generator=anchor_generator)
transforms = [
    DetectionDecode(),
    RandomSampleCrop(),
    DetectionResize((300, 300)),
    DetectionToPercentCoords(),
    AssignGTToDefaultBox(ssd_encoder),
    DetectionRandomColorAdjust(brightness=0.4, contrast=0.4, saturation=0.4),
    DetectionNormalize(mean=[0.485 * 255, 0.456 * 255, 0.406 * 255],
                       std=[0.229 * 255, 0.224 * 255, 0.225 * 255]),
    DetectionHWC2CHW()
]

dataset = COCODetection(data_url,
                        split="train",
                        transforms=transforms,
                        batch_size=64,
                        repeat_num=1,
                        num_parallel_workers=8,
                        shuffle=True,
                        remove_invalid_annotations=True,
                        filter_crowd_annotations=True,
                        trans_record=True)

dataset_train = dataset.run()
step_size = dataset_train.get_dataset_size()

# Create model.
network = ssd_mobilenet_v2(num_classes=81, pretrained=False)
network.to_float(ms.float16)

# Set lr scheduler.
lr = nn.cosine_decay_lr(min_lr=0.0, max_lr=0.1,
                        total_step=epoch_size * step_size, step_per_epoch=step_size,
                        decay_epoch=epoch_size)


# Define optimizer.
network_opt = nn.Momentum(network.trainable_params(), lr, 0.9)

# Define training wrapper.
network = TrainingWrapper(network, network_opt)

# Set the checkpoint config for the network.
ckpt_config = CheckpointConfig(
    save_checkpoint_steps=step_size,
    keep_checkpoint_max=10)
ckpt_callback = ModelCheckpoint(prefix='ssd_mobilenet_v2',
                                directory=ckpt_save_dir,
                                config=ckpt_config)
# Init the model.
model = Model(network)

# Begin to train.
model.train(epoch_size,
            dataset_train,
            callbacks=[ckpt_callback, LossMonitor(lr)],
            dataset_sink_mode=True)

### 模型验证

模型验证过程与训练过程相似。不同的是验证过程不需要设置优化器和损失函数，但是需要计算评价指标，具体代码如下所示：

In [ ]:
import os

from mindspore import context

from mindvision.msdetection.dataset import COCODetection
from mindvision.msdetection.models.ssd import ssd_mobilenet_v2
from mindvision.msdetection.models.detection_engine import SSDDetectionEngine
from mindvision.msdetection.models.utils.ssd_utils import apply_eval
from mindvision.msdetection.dataset.transforms import DetectionDecode, DetectionResize, DetectionNormalize, \
    DetectionHWC2CHW

data_url = './coco2017/'

context.set_context(mode=context.GRAPH_MODE,
                    device_target='GPU')

# Data Pipeline.
transforms = [
    DetectionDecode(),
    DetectionResize((300, 300)),
    DetectionNormalize(mean=[0.485 * 255, 0.456 * 255, 0.406 * 255],
                       std=[0.229 * 255, 0.224 * 255, 0.225 * 255]),
    DetectionHWC2CHW()
]
dataset = COCODetection(data_url,
                        split="val",
                        transforms=transforms,
                        batch_size=32,
                        num_parallel_workers=8,
                        remove_invalid_annotations=True,
                        filter_crowd_annotations=True,
                        trans_record=True)

dataset_eval = dataset.run()

# Create model.
network = ssd_mobilenet_v2(num_classes=81, pretrained=True)

network.set_train(False)

# create detection engine.
ann_file = os.path.join(data_url, "annotations", "instances_val2017.json")
detection_engine = SSDDetectionEngine(num_classes=81,
                                      ann_file=ann_file,
                                      min_score=0.1,
                                      nms_threshold=0.6,
                                      max_boxes=100)

apply_eval(network, dataset_eval, detection_engine)

## 总结

本案例完成了一个SSD模型在COCO数据集上进行训练，验证的过程，其中，对关键的SSD模型结构和原理进行了详细讲解。如需完整的源码可以参考[MindSpore Vision套件](https://gitee.com/mindspore/vision/tree/master/mindvision/msdetection)。

## 引用

[Paper](https://arxiv.org/abs/1512.02325):   Wei Liu, Dragomir Anguelov, Dumitru Erhan, Christian Szegedy, Scott Reed, Cheng-Yang Fu, Alexander C. Berg.European Conference on Computer Vision (ECCV), 2016 (In press).